## TOC:
* [Functions](#Functions)
* [Parameters](#Parameters)
* [Execute in serie](#Execute-in-serie)
* [Execute in parallel](#Execute-in-parallel)
* [Final function](#Final-functions)


## Functions <a class="anchor" id="Functions"></a>

In [ ]:
#import libraries
import os
import cv2
import math
import numpy
import pandas
import geopandas
import matplotlib.pyplot
import matplotlib.patches
import pysal
import pathlib
from PIL import Image
from scipy.spatial import Voronoi, voronoi_plot_2d
from scipy.spatial import distance
from esda import Moran, Geary
from libpysal.weights import Queen
import ipywidgets
from ipywidgets import interact, interactive, fixed, interact_manual
from skimage import restoration
import warnings
warnings.filterwarnings('ignore')
import random

run(number, reverse)
Starting function for image analysis. loops through all images inside the folder to be processed.
number: is an integer, decides which path in folders list is processed.
reverse: decides the order in which images are processed



In [ ]:
def run(number,reverse=False):
    #Starting function for image analysis. loops through all images inside the folder to be processed.
    '''
    number is a integer, decides which path in folders list is processed.
    reverse decides the order in which images are processed
    '''
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(number)
    x=0
    ## Posision de archivos al inicio
    for label in labels:
        imlist=[]
        imlist2=[]
        for (root,dirs,files) in os.walk(impath):#, topandas.wn=True):
            if showing==True:
                print(str(x)+" "+root)
                print("")
            for file in files:
                if not file.endswith(imlabel):
                    continue
                if label in file:
                    if showing==True:                print("image file: "+file)
                    imlist += [os.path.join(root,file)]
                if labelR in file:
                    if showing==True: print("image file 2: "+file)
                    imlist2 += [os.path.join(root,file)]
        if reverse==True:
            imlist.reverse()
            imlist2.reverse()
        Conds=[]
        for Folder in [x for x in os.listdir(impath) if os.path.isdir(os.path.join(impath,x))]:
            Conds += [Folder]
        print(Conds)
        con=number
        Cond=Conds[con]
        print(Cond)
        imlistCon=[f for f in imlist if Cond in f]
        imlist2Con=[f for f in imlist2 if Cond in f]
        #thr2_value = thr2_values[con]
        print(len(imlistCon))
        if labelR=="":
            equallists=False
        elif len(imlistCon)==len(imlist2Con):
            equallists=True
        else:
            equallists=False
        for fn in range(len(imlistCon)):
            print(str(fn)+"/"+str(len(imlistCon)))
            imn=imlistCon[fn]
            if equallists==True:#2
                imn2=imlist2Con[fn]
            else:
                imn2=""
            #check if file was already processed
            SaveCSV=csvSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
            pathlib.Path(SaveCSV.replace(SaveCSV.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
            if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
                if showing == True:
                    print("Already processed")
            else:
                image(imn,imn2,SaveCSV,con,Cond,fn,imlistCon,showing,impath,label,labelR,imlabel,BadLabel,
                      ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,
                      intersection_threshold,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,Grain,equallists,imlist2Con,cheackarea)

In [ ]:
def ShowImage(image,showing):
    #function to show images in the output
    '''
    image is a numpy array containing an image.
    Showing is a boolean to decide if an image will be shown in output or not.
    '''
    if showing==True:
        matplotlib.pyplot.imshow(image)
        matplotlib.pyplot.show()
def saveImage(image,imageSave,name):
    #function to save images
    '''
    image is a numpy array containing an image.
    imageSave is a string that contains the path to save the image.
    name is a string with the label to which the image will be save.
    '''
    if imageSave!="":
        try:
            pathlib.Path(imageSave.replace(imageSave.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
            matplotlib.pyplot.imsave(imageSave.split(".")[0]+name+".png",image)
        except AttributeError:
            print("")
            #print("AttributeError while saving image")

In [ ]:
def ProcessingFluo(im,Label,thr_value,thr_value2,showing=False,rad=50,imageSave=""):
    #function to process fluorecent images to obtain a threshdolded image for countour detection.
    '''
    im is a numpy array containing an image.
    thr_value and thr_value2 are floats used in the image threshold.
    Showing is a boolean to decide if an image will be shown in output or not.
    rad is a integer to define the radium size in pixels of the ball to be used in the background subtraction fo the image. must be bigger in pixels than the colonies expected.
    imageSave is a string that contains the path to save the image.
    '''
    imgray=cv2.cvtColor(im, cv2.COLOR_BGR2GRAY) #threshold
    ShowImage(imgray,showing)
    saveImage(imgray,imageSave,"-grey-"+Label)
    background=restoration.rolling_ball(imgray, radius=rad)
    ShowImage(background,showing)
    saveImage(background,imageSave,"-background-"+Label)
    imgray2=imgray-background
    ShowImage(imgray2,showing)
    saveImage(background,imageSave,"-background-"+Label)
    ret, thresh=cv2.threshold(imgray2,thr_value,thr_value2,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)
    colors, counts = numpy.unique(thresh.reshape(-1, 1),return_counts = True,axis = 0)
    if thresh.size==sum(counts):
        if showing==True:
            print("The colonies area is: "+str(counts[1])+" pixels of a total of "+str(thresh.size))
            print("The rest is: "+str(counts[0])+" pixels of a total of "+str(thresh.size))
        try:
            ColonyArea=(counts[1]/thresh.size)*100
        except IndexError:
            ColonyArea=0.0
    ShowImage(thresh,showing)
    saveImage(thresh,imageSave,"-thresholded-"+Label)
    return(thresh,ColonyArea)

In [ ]:
def ProcessingB(im2,labelR,thr2_value,thr2_value2,Grain=False,showing=False,rad=50,imageSave="",equallists=True):
    #function to process the bright field images to obtain a thresholded image for countour detection of holes.
    '''
    im2 is a numpy array containing an image.
    thr2_value and thr2_value2 are floats used in the image threshold.
    Showing is a boolean to decide if an image will be shown in output or not.
    rad is a integer to define the radium size in pixels of the ball to be used in the background subtraction fo the image. must be bigger in pixels than the colonies expected.
    imageSave is a string that contains the path to save the image.
    equallists is a boolean, its true if the amount of images of bright field channel equal the amount fo fluoroscent channel images.
    '''
    if equallists==True:#5
        imgray2 = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY) #threshold
        ShowImage(imgray2,showing)
        saveImage(imgray2,imageSave,"-grey-"+labelR)
        '''
        equ = cv2.equalizeHist(imgray2)
        ShowImage(equ,showing)
        ret,th = cv2.threshold(equ, 50, 255, 0)
        ShowImage(th,showing)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
        ShowImage(kernel,showing)
        dilate = cv2.morphologyEx(th, cv2.MORPH_DILATE, kernel,3)
        dilate = cv2.morphologyEx(dilate, cv2.MORPH_DILATE, kernel,3)
        dilate = cv2.morphologyEx(dilate, cv2.MORPH_DILATE, kernel,3)
        ShowImage(dilate,showing)
        '''
        ###
        ret, thresh2 = cv2.threshold(imgray2, thr2_value,thr2_value2,cv2.THRESH_BINARY)#+cv2.THRESH_OTSU)
        if Grain==True:
            thresh2=cv2.bitwise_not(thresh2)
        ShowImage(thresh2,showing)
        saveImage(thresh2,imageSave,"-thresholded-"+labelR)
        #ve el area de los agujeros
        colors, counts = numpy.unique(thresh2.reshape(-1, 1),return_counts = True,axis = 0)
        '''
        for numb in range(len(colors)):
            color=colors[numb]
            count=counts[numb]
            #print(str(color)+": "+str(count))
        '''
        if thresh2.size==sum(counts):
            if showing==True:
                if Grain==True:
                    print("Grain area is: "+str(counts[0])+" pixels of a total of "+str(thresh2.size))
                    print("Analized area is: "+str(counts[1])+" pixels of a total of "+str(thresh2.size))                    
                else:
                    print("Hole area is: "+str(counts[0])+" pixels of a total of "+str(thresh2.size))
                    print("Analized area is: "+str(counts[1])+" pixels of a total of "+str(thresh2.size))
        else:
            print("Hole area has a problem")
        try:
            AreaAnalized=(counts[1]/thresh2.size)*100
            holeArea=(counts[0]/thresh2.size)*100
        except IndexError:
            AreaAnalized=100
            holeArea=0
            #holeArea=(numpy.sum(thresh2)/100)/thresh2.size
            #holeArea=(numpy.sum(thresh)/100)/(thresh.size-numpy.sum(thresh2))
    else:
        holeArea=0
        thresh2=None
        AreaAnalized=1
    return(thresh2,holeArea,AreaAnalized)

In [ ]:
def ColonyContour(imn,Label,thresh,showing=False,imageSave=""):
    #funtion in which colony contours are obtained from the thresholded fluorescent image.
    '''
    imn is a string with the fluorescent image's path.
    Label is a string with which the image is labeled, it's usally the fluorescent dyes' name.
    thresh is a numpy array with contains the thresholded image.
    Showing is a boolean to decide if an image will be shown in output or not.
    imageSave is a string that contains the path to save the image.
    '''
    #imagen 1 contornos
    imc=matplotlib.pyplot.imread(imn)
    ShowImage(imc,showing)
    contours,_=cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    imc=cv2.drawContours(imc, contours, -1, (0,255,0), 3)
    ShowImage(imc,showing)
    saveImage(imc,imageSave,"-contours1-"+Label)
    return(contours,imc)

In [ ]:
def HoleContour(imn2,labelR,thresh2,squaredPixel,equallists=True,showing=False,imageSave=""):
    #funtion in which colony contours are obtained from the thresholded fluorescent image.
    '''
    imn2 is a string with the reflection brightfield image's path.
    labelR is a string with which the image is labeled, it's usually "Bright".
    thresh2 is a numpy array with contains the thresholded image.
    SquaredPixel is a integer that should be the total pixels in the original image to process.
    equallists is a boolean, its true if the amount of images of bright field channel equal the amount fo fluoroscent channel images.
    Showing is a boolean to decide if an image will be shown in output or not.
    imageSave is a string that contains the path to save the image.
    '''
    if equallists==True:#7
        im2c=matplotlib.pyplot.imread(imn2)
        ShowImage(im2c,showing)
        contours2,_=cv2.findContours(thresh2, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        lcnt2=len(contours2)
        #print(lcnt2)
        im2c=cv2.drawContours(im2c, contours2, -1, (0,255,0), 3)
        ShowImage(im2c,showing)
        saveImage(im2c,imageSave,"-contours2-"+labelR)
        #ve el area de los agujeros
        for cnt2 in range(lcnt2):
            contour2 = contours2[cnt2]
            area = cv2.contourArea(contour2)
    else:
        contours2=None
        im2c=None
    return(contours2,im2c)

In [ ]:
def filterContours(contours,maxArea,minArea,contours2,showing,cheackarea=True,equallists=True,intersection_threshold=0.5):
    #funtion in the colony contours are filtered by their overpossition with holes or their too big or small size.
    '''
    contours is a list of contours obtained from the findContours() function from the fluorescent channel image.
    maxArea is a float of the maximun area a colony is expected to have in pixels.
    minArea is a float of the minimun area a colony is expected to have in pixels.
    contours2 is a list of contours obtained from the findContours() function from the reflection brightfield channel image.
    Showing is a boolean to decide if an image will be shown in output or not.
    checkarea is a boolean to decide whether to check the minimun and maximun area to filter out some contours.
    equallists is a boolean, its true if the amount of images of bright field channel equal the amount fo fluoroscent channel images.
    '''
    #ahora filtra los contornos de la imagen 1 si tienen una tamaño menor que el area minimo o mayor que el area maxima o si se sobreponen con contornos de la imagen 2
    filtered_contours=[]
    trash_contours=[]
    #Iteramos sobre los contornos en contours
    lcnt1=len(contours)
    for cnt1 in range(lcnt1):
        #print("---------------------------------------------------------------------------------")
        #if showing==True: print("contour 1: "+str(cnt1))
        contour1=contours[cnt1]
        keep_contour1=True  # Variable para indicar si se debe mantener el contorno 1
        if cheackarea==True:
            area=cv2.contourArea(contour1)
            #print(str(maxArea)+" > "+str(area)+" > "+str(minArea))
            if maxArea<area or area<minArea:
                #print("Wont keep this contour")
                keep_contour1=False
                trash_contours.append(contour1)
                continue
        #if keep_contour1==True:
        #    #if showing==True:                    print("this contour will be kept")
        #print("Keep processing ---------------------------")
        if equallists==True:#8
            #for contour2 in contours2:
            lcnt2=len(contours2)
            for cnt2 in range(lcnt2):
                #if showing==True: print("contours 2:"+str(cnt2))
                contour2=contours2[cnt2]
                #make sure the area isnt too small
                area2 = cv2.contourArea(contour2)
                if area2 < math.pi*(2**2):#if the area is smaller than the minimun, continue
                    #print("the area is too small to matter")
                    continue
                # Encuentra la intersección entre los dos contornos
                #if showing==True: print("checking the intersection")
                #check if both contours are convex
                if cv2.isContourConvex(contour1) and cv2.isContourConvex(contour2):
                    #check intersection
                    intersection = cv2.intersectConvexConvex(contour1, contour2, handleNested=True) # MAKES IT CRASH (sometimes) !!!!!! (fixed once you make sure the contour is covex)
                    # Verifica si la intersección es válida
                    if intersection[0] == (0.0):
                        #print("no intersection")
                        continue
                    elif intersection[0] != (0.0):
                        #if intersection[0] (intersection_threshold):
                        if showing==True: print("the holes touch the colony, get rid of it")
                        keep_contour1 = False
                        trash_contours.append(contour1)
                        break
        if keep_contour1==True:
            filtered_contours.append(contour1)
    return(trash_contours,filtered_contours)

In [ ]:
def ColonyCharac(filtered_contours,holeArea,pixeltoum,showing=False):
    #Function to determine and list the characteristics of each colony detected in contours.
    '''
    filtered_contours is a list of contours of the colonies outside of holes an with area within the expected size.
    holeArea are floats of the expected area of each hole detected.
    pixeltoum is the side size of a pixel in micrometers.
    Showing is a boolean to decide if an image will be shown in output or not.
    '''
    NNs=None
    Positions,Majors,Minors,Angles,Areas,Perimeters,Circularities,AspectRats,Rounds=[],[],[],[],[],[],[],[],[]
    for ncnt in range(len(filtered_contours)):
        contour1=filtered_contours[ncnt]
        distances=[]
        for ncnt2 in range(len(filtered_contours)):
            (xc,yc),(minor,major),angle = (0,0),(0,0),0
            (xc2,yc2),(minor2,major2),angle2 = (0,0),(0,0),0
            if ncnt == ncnt2:
                continue
            cnt = filtered_contours[ncnt]
            if 5 > len(filtered_contours[ncnt]):
                #print("not enough countours to get an ellipse in the first colony")
                circle = cv2.minEnclosingCircle(cnt)
                (xc,yc),rad = circle
            else:
                ellipse = cv2.fitEllipse(cnt)
                (xc,yc),(minor,major),angle = ellipse
            cnt2 = filtered_contours[ncnt2]
            if 5 > len(filtered_contours[ncnt2]):
                #print("not enough countours to get an ellipse in the second colony")
                circle=cv2.minEnclosingCircle(cnt2)
                (xc2,yc2),rad = circle
            else:
                ellipse = cv2.fitEllipse(cnt2)
                (xc2,yc2),(minor2,major2),angle2 = ellipse
            #if showing == True:
            #    print("Position "+str(ncnt)+" :"+str((xc,yc))+" Postion "+str(ncnt2)+" :"+str((xc2,yc2)))
            #now check the distance between the two points, pitagoras
            distan = pixeltoum*math.sqrt((xc-xc2)**2+(yc-yc2)**2)
            '''
            if showing == True:
                print("the distance between this points is: "+str(distan)+" pixels or "+str(distan*pixeltoum)+" micrometers")
            '''
            distances.append(distan)
        if showing == True:
            try:
                print("The nearest neighbor is at: "+str(min(distances))+" μm")
            except:
                print("Error in the nearest neighbor postion")
        try:
            NNs=min(distances)
        except ValueError:
            NNs=None
        #obten otros parametros importantes del contorno
        circl=False
        (xc,yc),(minor,major),angle = (0,0),(0,0),0
        cnt = filtered_contours[ncnt]
        try:
            ellipse = cv2.fitEllipse(cnt)
            (xc,yc),(minor,major),angle = ellipse
        except  Exception as e:
            if showing ==True:
                print(e)
                print("not enough countours to get an ellipse in the first colony")
            circle = cv2.minEnclosingCircle(cnt)
            (xc,yc),major = circle
            minor=major
        Area = cv2.contourArea(cnt)  # Area of first contour
        Perimeter = cv2.arcLength(cnt, True)  # Perimeter of first contour 
        major=major*pixeltoum
        minor=minor*pixeltoum
        Area=Area*pixeltoum*pixeltoum
        Perimeter=Perimeter*pixeltoum
        #otros parametros
        try:
            Ciruclarity = 4*(math.pi)*(Area/(Perimeter**2))
        except  Exception as e:
            if showing==True: print(e)
            Ciruclarity=None
        AspectRatio = minor/major
        try:
            Roundness = 4*(Area/(((math.pi)*major)**2))
        except:
            Roundness = None
        try:
            xc,yc=int(xc),int(yc)
        except  Exception as e:
            if showing == True: print(e)
        if showing == True:
            print("Position:"+str([xc,yc]))
            print("Major: "+str(major)+" μm")
            print("Minor: "+str(minor)+" μm")
            print("Angle: "+str(angle)+" μm")
            print("Area: "+str(Area)+" μm^2")
            print("Perimeter: "+str(Perimeter)+" μm")
            print("Circularity: "+str(Ciruclarity))
            print("Aspect Ratio: "+str(AspectRatio))
            print("Roundness: "+str(Roundness))
        Positions.append([xc,yc])
        Majors.append(major)
        Minors.append(minor)
        Angles.append(angle)
        Areas.append(Area)
        Perimeters.append(Perimeter)
        Circularities.append(Ciruclarity)
        AspectRats.append(AspectRatio)
        Rounds.append(Roundness)
    PercentHole=[]
    for num in range(len(Positions)):
        PercentHole.append(holeArea)
    return(Positions,Majors,Minors,Angles,NNs,Areas,Perimeters,Circularities,AspectRats,Rounds,PercentHole)

In [ ]:
def image(imn,imn2,SaveCSV,con,Cond,fn,imlistCon,showing,impath,Label,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,Grain=False,equallists=False,imlist2Con="",cheackarea=True):
    #Function to process one single image.
    '''
    imn and imn2, are strings with the name and path of the images with fluorescent channel and brightfield channel.
    SaveCSV is a string with the path to save the tables per image as .csv files.
    con is a integer to of which number of which folder in being processed.
    Cond is a string of the name of the folder whose image are being processed.
    fn is a integer of the number of image of the folder being processed.
    imlistCon list of string in which all the names and paths of the images to process are.
    Showing is a boolean to decide if an image will be shown in output or not.
    impath is a string with the path in which all folders to be processed are.
    Label is a string to label all the fluorescent images to process, usually "DAPI".
    labelR is a string to label all the Brightfield images to process, usually "Bright".
    imlabel is a string to the label of the image format to process, usually ".tif"
    BadLabel is a string to optionally label to disregard all the image paths that contain it.
    ImageSaveFolder is a string that contains the path to output images in.
    csvSaveFolder is a string that contains the path to output .csv tables in.
    thr_value, thr_value2, thr2_value, thr2_value2 are floats to use in the thresholding of fluorescent and brightfield images during its processing.
    intersection_threshold is a float to decide the size intersection to disregard colonys that overlap with holes #currently disabled.
    AreaperField is the area of the field of each image analyzed in mm^2
    squaredPixel is the total of pixels of the images to analyze.
    pixeltoum is the lenght of a pixel in micrometers.
    minArea is the minimun expected colony size in pixels
    maxArea is the maximun expected colony size in pixels
    imlist2Con list of string in which all the names and paths of the brightfield images to process are.
    Showing is a boolean to decide if an image will be shown in output or not.
    '''    
    #showing=True
    
    im=matplotlib.pyplot.imread(imn)
    if equallists==True:#3
        im2=matplotlib.pyplot.imread(imn2)
    else:
        im2=None
    if ImageSaveFolder!="":
        imageSave=ImageSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
        if equallists==True:
            saveImage(im2,imageSave,"-image-"+labelR)
    else:
        imageSave=""
    ShowImage(im,showing)
    saveImage(im,imageSave,"-image-"+Label)
    if equallists==True:#4
        ShowImage(im2,showing)
        saveImage(image,imageSave,"-image-"+labelR)
    #Process image 1
    thresh,ColonyArea=ProcessingFluo(im,Label,thr_value,thr_value2,showing,50,imageSave)
    #process image 2
    thresh2,holeArea,AreaAnalized=ProcessingB(im2,labelR,thr2_value,thr2_value2,Grain,showing,50,imageSave,equallists)
    # elimina las colonias en zonas de agujeros o fuera de granos
    if equallists==True:
        thresh=cv2.bitwise_and(thresh,thresh2)
    if equallists==True:#4
        ShowImage(im,showing)
        saveImage(im,imageSave,"-FilteroutIm-"+Label)
    # obten los contornos desde las imagenes binarias
    contours,imc=ColonyContour(imn,Label,thresh,showing,imageSave)
    contours2,im2c=HoleContour(imn2,labelR,thresh2,squaredPixel,equallists,showing,imageSave)
    trash_contours,filtered_contours=filterContours(contours,maxArea,minArea,contours2,showing,cheackarea,equallists,intersection_threshold)
    #muestra imagenes                    
    #if showing == True:
    ####
    print("this countours wont be keep")
    imc=cv2.drawContours(imc,trash_contours, -1, (0,255,0), 3)
    imc=matplotlib.pyplot.imread(imn)
    if showing == True:
        ShowImage(imc,showing)
    saveImage(imc,imageSave,"-trashContours-"+Label)
    # filtered_contours ahora contiene los contornos de la imagen 1 después de eliminar las superposiciones
    print("this countours will be keep")
    imc1=matplotlib.pyplot.imread(imn)
    imc1=cv2.drawContours(imc1, filtered_contours, -1, (0,255,0), 3)
    if showing == True:
        ShowImage(imc1,showing)
    saveImage(imc1,imageSave,"-GoodContours-"+Label)
    ###
    Positions,Majors,Minors,Angles,NNs,Areas,Perimeters,Circularities,AspectRats,Rounds,PercentHole=ColonyCharac(filtered_contours,holeArea,pixeltoum,showing)
    data=pandas.DataFrame({"Position": Positions,"Major [μm]": Majors,"Minor [μm]": Minors,
                           "Angle [μm]": Angles,"Area [μm^2]": Areas,"Perimeter [μm]": Perimeters,
                           "Circularity": Circularities,"Aspect Ratio": AspectRats,"Roundness": Rounds,
                           "NearestNeighbor [μm]": NNs,"Hole Area [%]": PercentHole,"Total colonies Area [%]":ColonyArea})
    if showing==True: 
        print("data is going to be saved")
        print(SaveCSV.split(".")[0]+".csv")
        print(data.head(4))
    data.to_csv(SaveCSV.split(".")[0]+".csv", index=False)

In [ ]:
def run2(number,reverse=False):
    #second part of function for image analysis. loops through all images inside the folder to be processed. takes the process of each image and makes a summary file for each folder with the images global parameters.
    '''
    number is a integer, decides which path in folders list is processed.
    reverse decides the order in which images are processed
    '''
    x=0
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(number)
    ## Posision de archivos al inicio
    if os.path.exists(ImageSaveFolder)==False:
        os.makedirs(ImageSaveFolder)
    if os.path.exists(csvSaveFolder)==False:
        os.makedirs(csvSaveFolder)
    imlist=[]
    imlist2=[]
    for label in labels:
        for (root,dirs,files) in os.walk(impath):#, topandas.wn=True):
            if showing==True:
                print(str(x)+" "+root)
                print("")
            for file in files:
                if not file.endswith(imlabel):
                    continue
                if label in file:
                    if showing==True:                print("image file: "+file)
                    imlist += [os.path.join(root,file)]
                if labelR in file:
                    if showing==True: print("image file 2: "+file)
                    imlist2 += [os.path.join(root,file)]
        if reverse==True:
            imlist.reverse()
            imlist2.reverse()
        Conds=[]
        #imlist.reverse()
        #imlist2.reverse()
        for Folder in [x for x in os.listdir(impath) if os.path.isdir(os.path.join(impath,x))]:
            Conds += [Folder]
        print(Conds)
        con=number
        Cond=Conds[con]
        print(Cond)
        imlistCon=[f for f in imlist if Cond in f and BadLabel not in f]
        imlist2Con=[f for f in imlist2 if Cond in f and BadLabel not in f]
        #thr2_value = thr2_values[con]
        #print(len(imlistCon))
        #check if file was already processed
        proc=0
        noproc=0
        for fn in range(len(imlistCon)):
            SaveCSV=csvSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
            pathlib.Path(SaveCSV.replace(SaveCSV.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
            if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
                proc=proc+1
            else:
                noproc=noproc+1
        if showing == True:
            print(str(proc)+"/"+str(len(imlistCon)))
        #f proc==len(imlistCon):
            #continuar
        AvMajor,AvMinor,AvAngle,AvArea,AvPer,AvCir,AvAR,AvRo,AvNN=[],[],[],[],[],[],[],[],[]
        PercentHole,ColonyDensity=[],[]
        FieldArea=[]
        RipleyKs,MoranIs,GearyCs,MoranIIs,GearyCIs,MoranCorrSizes,MoranCorrSizesR=[],[],[],[],[],[],[]
        imlistNames=[]
        TotalColonies=[]
        for fn in range(len(imlistCon)):
            image2(imlistCon,fn,csvSaveFolder,Cond,ImageSaveFolder,AreaperField,showing)

In [ ]:
def image2(imlistCon,fn,csvSaveFolder,Cond,ImageSaveFolder,AreaperField,showing=False):
    #Function to process one single image to make a summary file witha single row per image.
    '''
    imlistCon list of string in which all the names and paths of the images to process are.
    fn is a integer of the number of image of the folder being processed.
    csvSaveFolder is a string that contains the path to output .csv tables in.
    Cond is a string of the name of the folder whose image are being processed.
    ImageSaveFolder is a string that contains the path to output images in.
    AreaperField is the area of the field of each image analyzed in mm^2
    Showing is a boolean to decide if an image will be shown in output or not.
    '''   
    r_max=600
    print(str(fn)+"/"+str(len(imlistCon)))
    imn=imlistCon[fn]
    SaveCSV=csvSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
    SaveCSV2=csvSaveFolder+"/"+Cond.replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
    if os.path.isfile(SaveCSV2.split('.')[0]+'.csv'):     ### checkear si la imagen ya fue analizada
        data3=pandas.read_csv(SaveCSV2.split(".")[0]+".csv")
        ProcessedImages=data3["File Name"].to_list()
    else:
        ProcessedImages=[]
    if any(x in ProcessedImages for x in [imn.split(".")[0].split("/")[-1].split("\\")[-1]]):
        print(imn.split(".")[0].split("/")[-1].split("\\")[-1]+"is already processed")
    else:
        pathlib.Path(SaveCSV.replace(SaveCSV.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
        if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
            print(SaveCSV.split(".")[0]+".csv")
            data=pandas.read_csv(SaveCSV.split(".")[0]+".csv")
        else:
            #print("this image hasnt been processed yet: "+SaveCSV.split(".")[0]+".csv".split("/")[-1])
            return()
        #print(SaveCSV)
        x,y,radius,coordinates=[],[],[],[]
        try:
            for n in range(len(data["Position"])):
                try:#1
                    if math.isnan(data["Position"][n].replace("(","").replace(")","").split(",")[0].replace("[","").replace("]","")) or math.isnan(y):
                        print("X or Y is NaN")
                        continue
                    x.append(int(data["Position"][n].replace("(","").replace(")","").split(",")[0].replace("[","").replace("]","")))
                    y.append(int(data["Position"][n].replace("(","").replace(")","").split(",")[1].replace("[","").replace("]","")))
                    radius.append(int(data["Major [μm]"][n]))
                    coor=[int(data["Position"][n].replace("(","").replace(")","").replace("[","").replace("]","").split(",")[0]),int(data["Position"][n].replace("(","").replace(")","").replace("[","").replace("]","").split(",")[1])]
                    coordinates.append(coor)
                except Exception as e:
                    print("error im2 1")
                    print("X")
                    print(data["Position"][n].replace("(","").replace(")","").split(",")[0].replace("[","").replace("]",""))
                    print("Y")
                    print(data["Position"][n].replace("(","").replace(")","").split(",")[1].replace("[","").replace("]",""))
                    print("rad")
                    print(data["Major [μm]"][n])
                    print(e)
                    continue
            coordinatesN = numpy.array(coordinates)
            #Voronoi
            if len(coordinatesN) > 2:
                vor = Voronoi(coordinatesN)
                fig = voronoi_plot_2d(vor)
                fig = voronoi_plot_2d(vor, show_vertices=False, line_colors='orange',line_width=2, line_alpha=0.6, point_size=2)
                if ImageSaveFolder != "":
                    matplotlib.pyplot.savefig(ImageSaveFolder+"/Voronoi-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
                if showing == True:
                    matplotlib.pyplot.show()
                else: matplotlib.pyplot.close()
            #Segundo Ripley's K
            if showing==True: print("ripleyK")
            r_values = numpy.linspace(0, r_max, num=100)  # You can adjust the number of bins
            n = len(coordinatesN)
            k_values = numpy.zeros(len(r_values))
            for i, r in enumerate(r_values):
                k_values[i] = numpy.sum([numpy.sum(numpy.logical_and(distance.cdist([p], coordinatesN) <= r, distance.cdist([p], coordinatesN) > 0)) - 1 for p in coordinatesN]) / n
            matplotlib.pyplot.plot(r_values, k_values, marker='o')
            matplotlib.pyplot.axline((0, 0), slope=1, color='red', linestyle='--', label='$K(r) = r$')  # Add the reference line
            matplotlib.pyplot.xlabel('Distance')
            matplotlib.pyplot.ylabel('Ripley\'s K-function')
            matplotlib.pyplot.legend()
            if ImageSaveFolder != "":
                matplotlib.pyplot.savefig(ImageSaveFolder+"/RipleyK-"+imlistCon[fn].split("\\")[-1].split("/")[-1].split(".")[0]+".png")
            if showing == True:
                matplotlib.pyplot.show()
            else: matplotlib.pyplot.close()
            max_k_index = numpy.argmax(k_values)  # Index of maximum K-value
            max_r = r_values[max_k_index]  # Distance at maximum K-value
            max_k = k_values[max_k_index]  # Maximum K-value
            if max_k < max_r:
                interpretation = "Clustering"  # K(r) < r suggests clustering
            elif max_k > max_r:
                interpretation = "Regular/Uniform"  # K(r) > r suggests regular or uniform distribution
            else:
                interpretation = "Random"  # K(r) = r suggests random distribution
            if showing == True:    print("Ripley's K:", interpretation)
            RipleyKs=interpretation
            # Create a GeoDataFrame from the colony data
            geometry = geopandas.points_from_xy(x, y)
            datag = {'radius': radius}  # Include other attributes if needed
            try:#2
                gdf = geopandas.GeoDataFrame(datag, geometry=geometry)
                # Create a spatial weights matrix (Queen contiguity)
                moran_interpretation = None
                geary_interpretation = None
                try:#3
                    w = Queen.from_dataframe(gdf)
                    if showing==True: print("moran and geary")
                    # Calculate Moran's I
                    moran = Moran(gdf.radius, w)
                    # Calculate Geary's C
                    geary = Geary(gdf.radius, w)
                    moran_value, geary_value=moran.I, geary.C
                    # Interpret Moran's I
                    if moran_value > 0:
                        moran_interpretation = "Spatial Clustering"
                    elif moran_value < 0:
                        moran_interpretation = "Spatial Dispersion"
                    else:
                        moran_interpretation = "Random Spatial Pattern"
                    # Interpret Geary's C
                    if geary_value < 0.5:
                        geary_interpretation = "Spatial Clustering"
                    elif geary_value > 0.5:
                        geary_interpretation = "Spatial Dispersion"
                    else:
                        geary_interpretation = "Random Spatial Pattern"
                    if showing == True:
                        # Print results
                        print("Moran's I:", moran.I)
                        print("Geary's C:", geary.C)
                        print("Interpretation of Moran's I:", moran_interpretation)
                        print("Interpretation of Geary's C:", geary_interpretation)
                    MoranI=moran.I
                    GearyC=geary.C
                    MoranII=moran_interpretation
                    GearyCI=geary_interpretation
                    size_list=radius
                    position_list=coordinates
                    #size_autocorrelation = calculate_size_autocorrelation(size_list, position_list, w)
                    size_list, position_list, spatial_weights = size_list, position_list, w
                    moran = Moran(size_list, spatial_weights)
                    size_autocorrelation=moran.I
                    if showing == True:
                        print("Spatial Autocorrelation of Colony Sizes (Moran's I):", size_autocorrelation)
                    MoranCorrSize=size_autocorrelation
                    moran_i_value =  size_autocorrelation  # Replace with your calculated Moran's I value
                    #interpretation = interpret_autocorrelation_result(moran_i_value)
                    if moran_i_value > 0:
                        interpretation = "Positive spatial autocorrelation: Larger sizes tend to be near other larger sizes."
                    elif moran_i_value < 0:
                        interpretation = "Negative spatial autocorrelation: Smaller sizes tend to be near larger sizes."
                    else:
                        interpretation = "No significant spatial autocorrelation: Colony sizes are randomly distributed."
                    if showing == True:
                        print("Interpretation of Spatial Autocorrelation Result:", interpretation)
                    MoranCorrSizeR=interpretation
                except:
                    MoranI=None
                    GearyC=None
                    MoranII=None
                    GearyCI=None
                    MoranCorrSize=None
                    MoranCorrSizeR=None
                try:#4
                    PercentHole=float(data["Hole Area 2 [%]"][0])/100
                    count=len(data)
                    Ncol=len(data["Position"].dropna())
                    FieldArea=((float(data['Total colonies Area [%]'][0])*AreaperField)/((1-PercentHole)*AreaperField))#AreaperField in mm^2
                except:
                    PercentHole=None
                    count=len(data)
                    Ncol=None
                    FieldArea=((float(data['Total colonies Area [%]'][0])))
                try:#5
                    AreaAn=((100-float(csv['Hole Area [%]'][0]))*AreaperField)/100
                except:
                    AreaAn=AreaperField
            except:
                print("error im2 2")
                MoranI=None
                GearyC=None
                MoranII=None
                GearyCI=None
                MoranCorrSize=None
                MoranCorrSizeR=None
                PercentHole=None
                count=len(data)
                Ncol=None
                try:
                    FieldArea=float(data['Total colonies Area [%]'][0])#AreaperField in mm^2, *100 para obtener un porcentaje
                except:
                    FieldArea=0.0
                AreaAn=AreaperField
            CDen=count/AreaAn
            print("colony density: "+str(CDen))
            data2=pandas.DataFrame({"File Name":imn.split(".")[0].split("/")[-1].split("\\")[-1],
                                    "Major mean [μm]": [data["Major [μm]"].mean(axis=0)],
                                    "Major std [μm]": [data["Major [μm]"].std(axis=0)],
                                    "Major N [μm]": [len(data["Major [μm]"])+1],
                                    "Minor mean [μm]": [data["Minor [μm]"].mean(axis=0)],
                                    "Minor std [μm]": [data["Minor [μm]"].std(axis=0)],
                                    "Minor N [μm]": [len(data["Minor [μm]"])+1],
                                    "Angle mean [μm]": [data["Angle [μm]"].mean(axis=0)],
                                    "Angle std [μm]": [data["Angle [μm]"].std(axis=0)],
                                    "Angle N [μm]": [len(data["Angle [μm]"])+1],
                                    "Colony area mean [μm^2]": [data["Area [μm^2]"].mean(axis=0)],
                                    "Colony area std [μm^2]": [data["Area [μm^2]"].std(axis=0)],
                                    "Colony area N [μm^2]": [len(data["Area [μm^2]"])+1],
                                    "Colony area sum [μm^2]": [sum(data["Area [μm^2]"])],
                                    "Perimeter mean [μm]": [data["Perimeter [μm]"].mean(axis=0)],
                                    "Perimeter std [μm]": [data["Perimeter [μm]"].std(axis=0)],
                                    "Perimeter N [μm]": [len(data["Perimeter [μm]"])+1],
                                    "Circularity mean": [data["Circularity"].mean(axis=0)],
                                    "Circularity std": [data["Circularity"].std(axis=0)],
                                    "Circularity N": [len(data["Circularity"])+1],
                                    "Aspect ratio mean": [data["Aspect Ratio"].mean(axis=0)],
                                    "Aspect ratio std": [data["Aspect Ratio"].std(axis=0)],
                                    "Aspect ratio N": [len(data["Aspect Ratio"])+1],
                                    "Roundness mean": [data["Roundness"].mean(axis=0)],
                                    "Roundness std": [data["Roundness"].std(axis=0)],
                                    "Roundness N": [len(data["Roundness"])+1],
                                    "Nearest Neighbor mean [μm]": [data["NearestNeighbor [μm]"].mean(axis=0)],
                                    "Nearest Neighbor std [μm]": [data["NearestNeighbor [μm]"].std(axis=0)],
                                    "Nearest Neighbor N [μm]": [len(data["NearestNeighbor [μm]"])+1],
                                    "Colonized area [%]": [FieldArea],
                                    "Ripley's K": [RipleyKs],
                                    "Moran's I": [MoranI],
                                    "Geary's C": [GearyC],
                                    "Moran's I's interpretation": [MoranII],
                                    "Geary's C interpretation": [GearyCI],
                                    "Radius correlation with position by Moran's I": [MoranCorrSize],
                                    "Radius correlation with position by Moran's I interpretation": [MoranCorrSizeR],
                                    "Hole percent 2":[PercentHole],
                                    "Colony Density":[CDen]})
            SaveCSV2=csvSaveFolder+"/"+Cond.replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
            SaveCSV2=SaveCSV2.split('.')[0]+'.csv'
            if os.path.isfile(SaveCSV2):
                print("data3")
                data3=pandas.read_csv(SaveCSV2)
                data3 = pandas.concat([data3, data2], ignore_index=True, sort=False)
                data3.to_csv(SaveCSV2, index=False)
            else:
                print("data2")
                data2.to_csv(SaveCSV2, index=False)
        except:
            print("skip")

In [ ]:
def VarCoef(dn=0):
    #Function to get the variation coefficient from colony density, all other processed must have been completed before this function can be run successfully.
    '''
    dn is a integer, it's to process more than one flourescent dye in the same set of images, otherwise it mus tbe 0.
    '''   
    Repetitions=25 #numero de veces repetidos #25
    Nmin=0
    NGroups = 3 #cuantos grupos se van a hacer
    Numbers = [1,2,3,6,9,12,18,27,36,72,96,120]
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(0)
    MainFolder=csvSaveFolder+"/"+(impath.split("/")[-1])
    SaveFolder=csvSaveFolder
    dye = labels[dn]
    if not os.path.isdir(SaveFolder+'/CV'):
      os.mkdir(SaveFolder+'/CV')
    SaveFolder = SaveFolder+'/CV/'
    Folders = [f for f in os.listdir(MainFolder) if os.path.isdir(os.path.join(MainFolder,f)) and 'Analysis' not in f and 'FilteredImages' not in f] # selecciona subcarpetas con la informacion
    Folders.sort()    
    for n in range(len(Folders)):
        print(Folders[n])
        Folder = Folders[n]
        Folder = Folders[n]
        csv_files = [f for f in os.listdir(os.path.join(MainFolder, Folder))]# if '.csv' in f]
        csv_files_data = [f for f in csv_files if 'FilteredImages' not in f and 'parametersUsed' not in f and 'ORG' not in f]
        path = os.path.join(MainFolder,Folder)
        if showing==True:
            print(csv_files_data)
            print(len(csv_files_data))
            print()
    #alternative with 3 groups
    for n in range(len(Folders)):
        Nums = Numbers
        print(Folders[n])
        Folder = Folders[n]
        #csv_files = [f for f in os.listdir(os.path.join(MainFolder, Folder))]
        csv_files=[]
        for path, directories, files in os.walk(os.path.join(MainFolder, Folder)):
            csv_files.extend([os.path.join(path,x) for x in files])
        csv_files = [x for x in csv_files if ".csv" in x]
        csv_files_data=csv_files
        print(len(csv_files))
        #print(csv_files)
        #Parametros para hacer grupos al azar
        Nmax = len(csv_files_data) #numero maximo en la lista de imagenes
        Nums = [n for n in Nums if (n*NGroups)<Nmax]
        for Num in Nums:
            print(Num)
            CCs_datas = pandas.DataFrame()
            #Cuantas veces se va a repetir
            for rep in range(Repetitions):
                CCs_data = pandas.DataFrame()
                #print('-'+str(rep))
                rang = range(Nmin, Nmax)
                Nlist = list(rang)
                ranNums = random.sample(Nlist, Num*NGroups)
                Gn = []
                for x in range(NGroups):
                    Gn.append(ranNums[(x)*Num:(x+1)*Num])
                for ranNums in Gn:
                    #print(ranNums)
                    csv_files = []
                    for n in ranNums:
                        csv_file = csv_files_data[n]
                        csv_files.append(csv_file)
                    #print(csv_files)
                    #print(len(csv_files))
                    ColonyCounts, AllColonies = pandas.DataFrame(), pandas.DataFrame()
                    for x in ranNums:
                        csv_file = csv_files_data[x]
                        try: #1
                            #print(csv_file)
                            csv = pandas.read_csv(csv_file).assign(filename = csv_file.split("\\")[-1])
                            try:#2
                                csv['count'] = len(csv)
                            except IndexError:
                                csv['count'] = 0.0
                            try:#3
                                last_row = csv.iloc[-1:]
                            except:
                                last_row = csv.iloc[-1:]
                            try:#4
                                last_row['count'] = len(csv)
                            except IndexError:
                                last_row['count'] = 0.0
                            try:
                                Total_Colony_Area = (228.86*170.95*(10**-6)-float(csv['Hole Area [%]'][0]))
                            except:
                                Total_Colony_Area = (228.86*170.95*(10**-6))
                            last_row['Total Colony Area'] = Total_Colony_Area
                        except pandas.errors.EmptyDataError:
                            #print('EmtyDataError')
                            print('')
                            continue
                        ColonyCounts = pandas.concat([ColonyCounts, last_row])
                        #AllColonies = AllColonies.append(csv)
                    ColonyCounts = ColonyCounts.reset_index(drop=True)
                    ColonyCount = ColonyCounts
                    #print(ColonyCount.head(2))
                    try:#5
                        ColonyCount['Area Analysed [mm^2]'] = ColonyCount['Total Area Analysed']*AreaperField
                    except KeyError:
                        ColonyCount['Area Analysed [mm^2]'] = AreaperField
                    try:#6
                        ColonyCount['Colony Density [mm^-2]'] = ColonyCount['count']/ColonyCount['Area Analysed [mm^2]']
                    except TypeError:
                        ColonyCount['count'] = ColonyCount['count'].replace('NaN',0)
                        ColonyCount['Colony Density [mm^-2]'] = ColonyCount['count']/ColonyCount['Area Analysed [mm^2]']
                    except KeyError:
                        ColonyCount['count'] = 0
                        ColonyCount['Colony Density [mm^-2]'] = ColonyCount['count']/ColonyCount['Area Analysed [mm^2]']
                    if ColonyCount['Area Analysed [mm^2]'].min() == 0:
                        ColonyCount['Colony Density [mm^-2]'].replace([np.inf, -np.inf], 0, inplace=True)            
                    CC_mean = ColonyCount['Colony Density [mm^-2]'].mean()
                    CC_std = ColonyCount['Colony Density [mm^-2]'].std()
                    #print('mean','std')
                    #print(CC_mean,CC_std)
                    CC_data = pandas.DataFrame(data={'Folder':[Folder],'rep':[rep],'size':[Num],'mean':[CC_mean]})#,'std':[CC_std]})
                    #CCs_data = CCs_data.append(CC_data)
                    CCs_data = pandas.concat([CCs_data, CC_data])
                CCs_std = CCs_data['mean'].std()
                CCs_mean = CCs_data['mean'].mean()
                CCs_data = pandas.DataFrame(data={'Folder':[Folder],'rep':[rep],'size':[Num],'mean':[CCs_mean],'std':[CCs_std]})#,'std':[CC_std]})
                #CCs_datas = CCs_datas.append(CCs_data)
                CCs_datas = pandas.concat([CCs_datas, CCs_data])
            CCs_datas['CV'] = (CCs_datas['std']/CCs_datas['mean'])*100
            CCs_datas.to_csv(SaveFolder+'CountCV'+'_rep_'+str(rep)+'_size_'+str(Num)+'_sample_'+Folder+'_'+MainFolder.split('/')[-1].replace('.','')+'_'+MainFolder.split('/')[-2].replace('.','')+'.csv')

In [ ]:
### Posision de archivos al inicio
def FilePositions():
    #Function to get the folders to analyse afterwards
    x=0
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(0)
    label=labels[0]
    imlist=[]
    imlist2=[]
    for (root,dirs,files) in os.walk(impath): #, topandas.wn=True):
        if showing==True:
            print(str(x)+" "+root)
            print("")
        for file in files:
            if not file.endswith(imlabel):
                continue
            if label in file:
                if showing==True:                print("image file: "+file)
                imlist += [os.path.join(root,file)]
            if labelR in file:
                if showing==True: print("image file 2: "+file)
                imlist2 += [os.path.join(root,file)]
    
    Conds=[]
    for Folder in [x for x in os.listdir(impath) if os.path.isdir(os.path.join(impath,x))]:
        Conds += [Folder]
    print(Conds)
    return(Conds,imlist,imlist2)

In [ ]:
def checkProcessProgress():
    #Function to check the progress of analyzed images in run(), shows how many images have an .csv ready.
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(0)
    Conds,imlist,imlist2=FilePositions()
    #showing=True
    for con in range(len(Conds)):
        Cond=Conds[con]
        #print(Cond)
        imlistCon=[f for f in imlist if Cond in f]
        #print(len(imlistCon))
        proc=0
        noproc=0
        for fn in range(len(imlistCon)):
            SaveCSV=csvSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
            pathlib.Path(SaveCSV.replace(SaveCSV.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
            if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
                proc=proc+1
            else:
                noproc=noproc+1
        #if showing == True:
        print(str(con)+"-"+Cond+": "+str(proc)+"/"+str(len(imlistCon)))
        #f proc==len(imlistCon):
            #continuar

In [ ]:
def checkProcess2Progress():
    #Function to check the progress of analyzed images in run2(), shows how many rows the summary file has.
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(0)
    Conds,imlist,imlist2=FilePositions()
    for con in range(len(Conds)):
        #con=0
        Cond=Conds[con]
        print(Cond)
        imlistCon=[f for f in imlist if Cond in f]
        #print(len(imlistCon))
        #check if file was already processed
        proc=0
        noproc=0
        SaveCSV=csvSaveFolder+"/"+Cond.replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
        file = pandas.read_csv(SaveCSV.split(".")[0]+".csv")
        #print(len(file))
        file_noduplicates = file.drop_duplicates(subset=['File Name'], keep='first')
        print(len(file_noduplicates))
        file_noduplicates.to_csv(SaveCSV.split(".")[0]+".csv", index=False)
        file_noduplicates
        print()

In [ ]:
def ReorderTables():
    #Function to get another summary csv tables, this time one table per parameter obtained.
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(0)
    Conds,imlist,imlist2=FilePositions()
    csvFolder=csvSaveFolder
    FoldersAll=[os.path.join(csvFolder,f) for f in os.listdir(csvFolder) if f.endswith("-All.csv")]
    FolderAll=FoldersAll[0]
    data=pandas.read_csv(FolderAll)
    colNames=data.columns.values
    ColName=colNames[0]
    ################
    for n2 in range(len(colNames)):
        #n2=0
        ColName=colNames[n2]
        #nuevo dataframe para la columna
        DataC=pandas.DataFrame()
        for n1 in range(len(FoldersAll)):
            try:
                #1=0
                FolderAll=FoldersAll[n1]
                ConName=FolderAll.split("\\")[-1].replace("-All.csv","")
                data=pandas.read_csv(FolderAll)
                #data=data.sort_values(by=['Minor [μm]'])
                col=data[ColName]
                #col=[x for x in col if "Unnamed" not in x]
                #print(ConName)
                #print(ColName)
                #print(col)
                DataC[ConName]=col
            except:
                continue
        print(os.path.join(csvFolder,ColName+".csv"))
        DataC.to_csv(os.path.join(csvFolder,ColName+".csv"))
    FoldersAll=[os.path.join(csvFolder,f.replace("-All","")) for f in os.listdir(csvFolder) if f.endswith("-All.csv")]
    print(FoldersAll)
    FolderAll=FoldersAll[0]
    data=pandas.read_csv(FolderAll)
    colNames=data.columns.values
    ColName=colNames[0]

    ################
    for n2 in range(len(colNames)):
        #n2=0
        ColName=colNames[n2]
        #nuevo dataframe para la columna
        DataC=pandas.DataFrame()
        for n1 in range(len(FoldersAll)):
            try:
                #1=0
                FolderAll=FoldersAll[n1]
                ConName=FolderAll.split("\\")[-1].replace("-All.csv","")
                data=pandas.read_csv(FolderAll)
                #data=data.sort_values(by=['Minor mean [μm]'])
                col=data[ColName]
                #col=[x for x in col if "Unnamed" not in x]
                #print(ConName)
                #print(ColName)
                #print(col)
                DataC[ConName]=col
            except:
                continue
        print(os.path.join(csvFolder,ColName+".csv"))
        DataC.to_csv(os.path.join(csvFolder,ColName+".csv"))

In [ ]:
def run3(number,reverse=False):
    #Starting function for image analysis. loops through all images inside the folder to be processed.
    '''
    number is a integer, decides which path in folders list is processed.
    reverse decides the order in which images are processed
    '''
    x=0
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(number)
    ## Posision de archivos al inicio
    for label in labels:
        imlist=[]
        imlist2=[]
        for (root,dirs,files) in os.walk(impath):#, topandas.wn=True):
            if showing==True:
                print(str(x)+" "+root)
                print("")
            for file in files:
                if not file.endswith(imlabel):
                    continue
                if label in file:
                    if showing==True:                print("image file: "+file)
                    imlist += [os.path.join(root,file)]
                if labelR in file:
                    if showing==True: print("image file 2: "+file)
                    imlist2 += [os.path.join(root,file)]
        if reverse==True:
            imlist.reverse()
            imlist2.reverse()
        Conds=[]
        for Folder in [x for x in os.listdir(impath) if os.path.isdir(os.path.join(impath,x))]:
            Conds += [Folder]
        print(Conds)
        #for con in range(len(Conds)):
        con=number
        Cond=Conds[con]
        print(Cond)
        imlistCon=[f for f in imlist if Cond in f]
        #imlist2Con=[f for f in imlist2 if Cond in f]
        #thr2_value = thr2_values[con]
        #print(len(imlistCon))
        #check if file was already processed
        proc=0
        noproc=0
        datas=pandas.DataFrame()
        for fn in range(len(imlistCon)):
            print(str(fn)+"/"+str(len(imlistCon)))
            imn=imlistCon[fn]
            #imn2=imlist2Con[fn]
            SaveCSV=csvSaveFolder+"/"+imlistCon[fn].replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")
            pathlib.Path(SaveCSV.replace(SaveCSV.split("/")[-1],"")).mkdir(parents=True, exist_ok=True)
            if os.path.isfile(SaveCSV.split('.')[0]+'.csv'):
                print("Opening:")
                print(SaveCSV.split("/")[-1])
                try:
                    data = pandas.read_csv(SaveCSV.split(".")[0]+".csv")
                    datas = pandas.concat([datas, data], ignore_index=True, sort=False)
                except:
                    print("couldnt open file")
        SaveCSV=csvSaveFolder+"/"+Cond.replace("\\","/").replace(csvSaveFolder.replace(csvSaveFolder.split("/")[-1],""),"")+"-All"
        print(SaveCSV)
        datas.to_csv(SaveCSV.split(".")[0]+".csv", index=False)

In [ ]:
'''
def CVreor():
    #read the csv files
    FolderCV= "C:/Users/veralab/ImageAnalysis/JavieraP/CV"
    Folds = os.listdir(FolderCV)
    for Fold in Folds:
        Fold=os.path.join(FolderCV,Fold)
        Fold2s = [x for x in os.listdir(Fold) if os.path.isdir(os.path.join(Fold,x))]
        for Fold2 in Fold2s:
            Fold2=os.path.join(Fold,Fold2)
            Files=[x for x in os.listdir(Fold2) if x.endswith(".csv")]
            #check size_x, check the value right before .csv
            for File in Files:
                print(File)
                size=File.split("_")[4].replace(".csv","")
                FilName = "_".join(File.split("_")[6:]).replace(".csv","").replace("-","_")
                Strain = Fold2.split("\\")[1].replace(".csv","")
                Cond = Fold2.split("\\")[2].replace(".csv","")
                Renaming = "-".join([Strain,Cond,size,FilName])
                print(Renaming)
                try:
                    print(Renaming)
                    os.rename(os.path.join(Fold2,File),os.path.join(Fold2,Renaming+".csv"))
                except FileExistsError:
                    print("File exists error")
    #FolderCV= "D:/CuponesPirita/cv2sp3"
    CVTable={}
    size=[str(x) for x in [1,2,3,6,9,12,18,36,72,96,120]]
    #read the csv files
    Folds = [x for x in os.listdir(FolderCV) if os.path.isdir(os.path.join(FolderCV,x))]
    print(Stra)
    print(Con)
    for Fold in Folds:
        Fold=os.path.join(FolderCV,Fold)
        Fold2s = [x for x in os.listdir(Fold) if os.path.isdir(os.path.join(Fold,x))]
        print(Fold2s)
        for Fold2 in Fold2s:
            try:
                Fold2=os.path.join(Fold,Fold2)
                Files=[x for x in os.listdir(Fold2) if x.endswith(".csv")]
            except:
                continue
            #check size_x, check the value right before .csv
            for File in Files:
                #print(File)
                Strain = File.split("-")[0]
                Cond = File.split("-")[1]
                size=File.split("-")[2]
                FilName = "-".join(File.split("-")[3:]).replace(".csv","")
                Renaming = "-".join([Strain,Cond,size,FilName])
                print(Renaming)
                if Stra==Strain and Cond==Con:
                    FilPD=pd.read_csv(os.path.join(Fold2,File))
                    #print(FilPD)
                    CVTable[(Strain,Cond,FilName,size)]=FilPD["CV"].tolist()
                    #print(size)
                    #print(FilName)
                    #print(CVTable[(FilName,size)])
    print(sorted(list(CVTable.keys())))  
    print(CVTable)
    FilNames=list(set([x[2] for x in sorted(list(CVTable.keys()))]))
    for FName in FilNames:
        print(FName)
        NewCSV = os.path.join(FolderCV,Stra+"-"+C+"-"+FName+".csv")
        print(NewCSV)
        NewDat={}
        for size in sizes:
            try:
                print(CVTable[(Stra,C,FName,size)])
                NewDat[size]=CVTable[(Stra,C,FName,size)]
                print(size)
            except KeyError:
                print("KeyError")
                pass
        NewDat=pd.DataFrame.from_dict(NewDat, orient="index")
        if len(NewDat) > 0:
            print(NewCSV)
            NewDat.to_csv(NewCSV, sep=',', index=True, encoding='utf-8')
'''

In [ ]:
#read the csv files
def CVReorder():
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,saveCSV,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(0)
    #showing=True
    FolderCV=saveCSV+"/CV"
    sizes=[str(x) for x in [1,2,3,6,9,12,18,36,72,96,120]]
    Folds=os.listdir(FolderCV)
    Files=[x for x in os.listdir(FolderCV) if x.endswith(".csv")]
    for File in Files:
        print("File")
        print(File)
        size=File.split("_")[4].replace(".csv","")
        #size=File.split("_")[0].replace(".csv","")
        FilName = "_".join(File.split("_")[6:]).replace(".csv","")
        FilName = "_".join(File.split("_")[0:]).replace(".csv","")
        Renaming = "-".join([size,FilName])
        print("Renaming1")
        print(Renaming)
        try:
            os.rename(os.path.join(FolderCV,File),os.path.join(FolderCV,Renaming+".csv"))
        except FileExistsError:
            print("File exists error")
    CVTable={}
    Files=[x for x in os.listdir(FolderCV) if x.endswith(".csv")]
    #check size_x, check the value right before .csv
    for File in Files:
        print("File2")
        print(File)
        size=File.split("-")[0]
        #FilName = "-".join(File.split("-")[1:3]).replace(".csv","")
        FilName = "-".join(File.split("-")[1:6]).replace(".csv","")
        #Renaming = "-".join([size,FilName])
        #print("Renaming")
        #print(Renaming)
        FilPD=pandas.read_csv(os.path.join(FolderCV,File))
        print("FilPD")
        print(FilPD)
        CVTable[(size,FilName)]=FilPD["CV"].tolist()
    #print(sorted(list(CVTable.keys())))
    #print(CVTable)
    print("CVTable keys")
    print(CVTable.keys())
    FilNames=list(set([x[1] for x in sorted(list(CVTable.keys()))]))
    print(FilNames)
    for FName in FilNames:
        print("FName")
        print(FName)
        NewCSV = os.path.join(saveCSV,"CV-"+FName+".csv")
        print("NewCSV")
        print(NewCSV)
        NewDat={}
        for size in sizes:
            #try:
            NewDat[size]=CVTable[(size,FName)]
            print("Size")
            print(size)
            #except KeyError:
            #    print("KeyError")
            #    pass
        NewDat=pandas.DataFrame.from_dict(NewDat, orient="index")
        if len(NewDat) > 0:
            print("NewCSV")
            print(NewCSV)
            NewDat.to_csv(NewCSV, sep=',', index=True, encoding='utf-8')

In [ ]:
def CVReorder():
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,saveCSV,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(0)
    #showing=True
    FolderCV=saveCSV+"/CV"
    sizes=[str(x) for x in [1,2,3,6,9,12,18,36,72,96,120]]
    Folds=os.listdir(FolderCV)
    Files=[x for x in os.listdir(FolderCV) if x.endswith(".csv")]
    for File in Files:
        print(File)
        size=File.split("_")[4].replace(".csv","")
        #size=File.split("_")[0].replace(".csv","")
        FilName = "_".join(File.split("_")[6:]).replace(".csv","")
        Renaming = "-".join([size,FilName])
        print("Renaming1")
        print(Renaming)
        try:
            os.rename(os.path.join(FolderCV,File),os.path.join(FolderCV,Renaming+".csv"))
        except FileExistsError:
            print("File exists error")

In [ ]:
def CVReorder2():
    #read the csv files
    showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,saveCSV,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain=Parameters(0)
    #showing=True
    FolderCV=saveCSV+"/CV"
    sizes=[str(x) for x in [1,2,3,6,9,12,18,36,72,96,120]]
    Folds=os.listdir(FolderCV)
    CVTable={}
    Files=[x for x in os.listdir(FolderCV) if x.endswith(".csv")]
    #check size_x, check the value right before .csv
    for File in Files:
        print("File2")
        print(File)
        size=File.split("-")[0]
        print("size:")
        print(size)
        #FilName = "-".join(File.split("-")[1:3]).replace(".csv","")
        FilName = "-".join(File.split("-")[1:6]).replace(".csv","")
        #Renaming = "-".join([size,FilName])
        #print("Renaming")
        #print(Renaming)
        FilPD=pandas.read_csv(os.path.join(FolderCV,File))
        #print("FilPD")
        #print(FilPD)
        CVTable[(size,FilName)]=FilPD["CV"].tolist()
    #print("CVTable")
    #print(CVTable)
    #print(sorted(list(CVTable.keys())))
    #print(CVTable)
    #print("CVTable keys")
    #print(CVTable.keys())
    FilNames=list(set([x[1] for x in sorted(list(CVTable.keys()))]))
    print(FilNames)
    for FName in FilNames:
        #print("FName")
        #print(FName)
        NewCSV = os.path.join(saveCSV,"CV-"+FName+".csv")
        print("NewCSV")
        print(NewCSV)
        NewDat={}
        for size in sizes:
            try:
                NewDat[size]=CVTable[(size,FName)]
                print("Size")
                print(size)
            except KeyError:
                print("KeyError")
                pass
        NewDat=pandas.DataFrame.from_dict(NewDat, orient="index")
        if len(NewDat) > 0:
            print("NewCSV")
            print(NewCSV)
            NewDat.to_csv(NewCSV, sep=',', index=True, encoding='utf-8')
            print(NewDat)
    print("-")

## Parameters <a class="anchor" id="Parameters"></a>

In [ ]:
def Parameters(number=0):
    #warnings.filterwarnings('ignore')
    BadLabel="_ORG"
    impath="path"
    labels=["dye"]
    labelR="Bright"
    imlabel=".tif"
    invertBF=False
    ImageSaveFolder="path"
    csvSaveFolder="path"
    if os.path.exists(ImageSaveFolder)==False:
        os.makedirs(ImageSaveFolder)
    if os.path.exists(csvSaveFolder)==False:
        os.makedirs(csvSaveFolder)
    pixeltoum = 0.178 #micrometros
    AreaperField = 228.86*170.95*(10**-6) #area de cada campo en mm^2
    squaredPixel = 1038*1388 #pixels
    thr_value = 20
    thr_value2 = 250
    thr2_value=250
    thr2_value2=250
    intersection_threshold = 0.5  # Ajusta este valor según tu necesidad
    x=0
    showing=False
    pixeltoum = 0.178 #micrometros
    minArea=(2**2)*math.pi
    maxArea=math.pi*50**2
    cheackarea=False
    Grain=False
    return(showing,impath,labels,labelR,imlabel,BadLabel,ImageSaveFolder,csvSaveFolder,thr_value,thr_value2,thr2_value,thr2_value2,intersection_threshold,pixeltoum,AreaperField,squaredPixel,pixeltoum,minArea,maxArea,cheackarea,Grain)

In [ ]:
Conds,imlist,imlist2=FilePositions()

In [ ]:
# folders to run
torun=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,
       21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,35,36,37,38,39,40,
       41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,
       61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,
       81,82,83,84,85,86,87,89,88,89,90,91,92,93,94,95,96,97,98,99,100,
       101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,
       121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,135,136,137,138,139,140,
       141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,
       161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,
       181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200]
print("this folders will be run with this parameters:")
for n in torun:
    print(n)
    print(Conds[n])

## Execute in parallel<a class="anchor" id="Execute-in-parallel"></a>

In [ ]:
#Ejecuta una sola vez
i = 0
totC = 12
i*totC+totC

In [ ]:
#FoldersC son las carpetas a ejecutar en cluster paralelos.
running = torun[i*totC:i*totC+totC]
totC = len(running)
i = i+1
running

In [ ]:
# Abre Clusters paralelos
import ipyparallel as ipp
cluster = ipp.Cluster(n=totC)
cluster.start_cluster_sync()
rc = cluster.connect_client_sync()
rc.wait_for_engines(n=totC)
print(rc.ids)

In [ ]:
# importa las librerias en cada cluster
with rc[:].sync_imports():
    import os
    import cv2
    import math
    import numpy
    import pandas
    import geopandas
    import matplotlib.pyplot
    import matplotlib.patches
    import pysal
    import pathlib
    from PIL import Image
    from scipy.spatial import Voronoi, voronoi_plot_2d
    from scipy.spatial import distance
    from esda import Moran, Geary
    from libpysal.weights import Queen
    import ipywidgets
    from ipywidgets import interact, interactive, fixed, interact_manual
    from skimage import restoration
    import warnings
    import matplotlib
    %matplotlib inline

In [ ]:
# Define las funciones enc ada cluster
rc[:].push(dict(run=run,
                run2=run2,
                Parameters=Parameters,
                ShowImage=ShowImage,
                saveImage=saveImage,
                ProcessingFluo=ProcessingFluo,
                ProcessingB=ProcessingB,
                ColonyContour=ColonyContour,
                HoleContour=HoleContour,
                filterContours=filterContours,
                ColonyCharac=ColonyCharac,
                image=image,
                image2=image2))

In [ ]:
#Ejecuta en paralelo
results = {}
for numb in range(len(running)):
    results[numb] = rc[numb].apply_async(run, running[numb])#, True)

In [ ]:
#Indica si los resultados estan listos. ejecuta cuantas veces sea necesario
for numb in range(totC):
    print('is engine '+str(numb)+' ready?:')
    print(results[numb].ready())
    print('')

In [ ]:
#Revisa si ocurrio algun error
for num in range(totC):
    print('Engine '+str(num)+' ')
    try:
        results[num].get(1)
        print('Results ready without error')
        print('')
    except Exception as error:
        print(error)
        print('')

In [ ]:
checkProcessProgress()

In [ ]:
(

In [ ]:
#Ejecuta en paralelo
results = {}
for numb in range(totC):
    results[numb] = rc[numb].apply_async(run2, running[numb])

In [ ]:
#Indica si los resultados estan listos. ejecuta cuantas veces sea necesario
for numb in range(totC):
    print('is engine '+str(numb)+' ready?:')
    print(results[numb].ready())
    print('')

In [ ]:
#Revisa si ocurrio algun error
for num in range(totC):
    print('Engine '+str(num)+' ')
    try:
        results[num].get(1)
        print('Results ready without error')
        print('')
    except Exception as error:
        print(error)
        print('')

In [ ]:
checkProcess2Progress()

## Execute in serie<a class="anchor" id="Execute-in-serie"></a>

In [ ]:
for n in reversed(torun):
    print(n)
    checkProcessProgress()
    run(n,True)
    checkProcessProgress()

In [ ]:
for n in reversed(torun):
    run2(n,True)
    checkProcess2Progress()

## Final functions<a class="anchor" id="#Final-functions"></a>

In [ ]:
for n in torun:
    run3(n)

In [ ]:
ReorderTables()

In [ ]:
VarCoef()

In [ ]:
CVReorder()

In [ ]:
CVReorder2()